<a href="https://colab.research.google.com/github/michel-wue/Seminar/blob/Tries_API_Data_extraction/Data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [127]:
import requests as r
import pandas as pd
from pandas import json_normalize
import re

In [111]:
Data_basic = pd.DataFrame()

In [115]:
Data_basic = pd.DataFrame(columns=['GameID','season','types','week','HomeTeam_ID','AwayTeam_ID','Winner'])
week = list(range(1, 19))
seasons=list(range(2017,2023))

In [114]:
seasons

[2015, 2016, 2017, 2018, 2019, 2020, 2021, 2022]

In [116]:

def infos(season,types,weeks):
  url = f"https://sports.core.api.espn.com/v2/sports/football/leagues/nfl/seasons/{season}/types/{types}/weeks/{weeks}/events"

  response = r.get(url)
  json_data = response.json()
  df = json_normalize(json_data['items'])
  for game in range(len(df)):
    l=[]
    response = r.get(df['$ref'][game])
    data=response.json()
    try:
      df2=json_normalize(data['competitions'])
      l.append(df2['id'][0])
      l.append(season)
      l.append(types)
      l.append(weeks)
      l.append(df2['competitors'][0][0]['id'])
      l.append(df2['competitors'][0][1]['id'])
      try:
        if df2['competitors'][0][0]['winner'] == True:
          l.append('Home')
        else:
          l.append('Away')
      except KeyError:
        l.append('NaN')
    except KeyError:
      print('it doesnt worked')
    Data_basic.loc[len(Data_basic)] = l


In [117]:
for x in seasons:  
  for i in week:
    infos(x,2,i)

In [118]:
Data_basic


,GameID,season,types,week,HomeTeam_ID,AwayTeam_ID,Winner
0,400951566,2017,2,1,17,12,Away
1,400951567,2017,2,1,2,20,Home
2,400951570,2017,2,1,3,1,Away
3,400951572,2017,2,1,4,33,Away
4,400951574,2017,2,1,5,23,Away
...,...,...,...,...,...,...,...
1564,401437957,2022,2,18,21,19,Home
1565,401437962,2022,2,18,25,22,Home
1566,401437963,2022,2,18,26,14,Home
1567,401437959,2022,2,18,28,6,Home


In [135]:
type(Data_basic['HomeTeam_ID'][0])





0    17
Name: HomeTeam_ID, dtype: object

In [121]:
GameID='400951566'
url=f'http://sports.core.api.espn.com/v2/sports/football/leagues/nfl/events/{GameID}/competitions/{GameID}/drives?lang=en&region=us'
response_drive = r.get(url)
drive_data = response_drive.json()
drive_df = json_normalize(drive_data['items'])

In [159]:
drive_df['result']


0              TD
1          FUMBLE
2           DOWNS
3              TD
4              FG
5            PUNT
6              TD
7            PUNT
8            PUNT
9              TD
10    END OF HALF
11           PUNT
12           PUNT
13             TD
14           PUNT
15           PUNT
16             TD
17           PUNT
18             FG
19             TD
20          DOWNS
21           PUNT
22           PUNT
23             TD
24           PUNT
Name: result, dtype: object

In [138]:
for i in drive_df:
  print(i)

$ref
id
description
sequenceNumber
yards
isScore
offensivePlays
result
shortDisplayResult
displayResult
team.$ref
endTeam.$ref
start.period.type
start.period.number
start.clock.value
start.clock.displayValue
start.yardLine
start.text
end.period.type
end.period.number
end.clock.value
end.clock.displayValue
end.yardLine
end.text
timeElapsed.value
timeElapsed.displayValue
source.id
source.description
plays.$ref
plays.count
plays.pageIndex
plays.pageSize
plays.pageCount
plays.items


In [150]:
Hometeam=Data_basic[Data_basic['GameID'] == GameID]['HomeTeam_ID']

In [153]:
Hometeam[0]

'17'

In [162]:
yards_Home=0
yards_Away=0
TD_Home=0
TD_Away=0
Data_basic[Data_basic['GameID'] == GameID]['HomeTeam_ID']
#if re.search(r"teams/(\d+)", drive_df['team.$ref'][0]).group(1) == Data_basic[Data_basic['GameID'] == GameID]['HomeTeam_ID']:
for index, row in drive_df.iterrows():
    if re.search(r"teams/(\d+)",row['team.$ref']).group(1) == Data_basic[Data_basic['GameID'] == GameID]['HomeTeam_ID'][0]:
      yards_Home=yards_Home + row['yards']
      if row['result']=='TD':
        TD_Home=TD_Home+1

    else:
      yards_Away=yards_Away + row['yards'] 
      if row['result']=='TD':
        TD_Away=TD_Away+1
        




In [161]:
TD_Home

3

In [163]:
TD_Away

5